# Import necessary packages

In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [5]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [6]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [7]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
565,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,...,1,0,1,1,1,0,0,0,0,1
566,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,1,2,4,...,0,0,0,0,0,1,0,0,0,1
567,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,1,2,1,0,0,0,1,0
568,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,4,0,8,11,...,0,5,0,0,0,0,0,0,1,0
569,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,3,2,2,...,1,0,3,1,2,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10754,HOUSEHOLD_2_512_CA_4_validation,HOUSEHOLD_2_512,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,2,1,0,0,0,0
10755,HOUSEHOLD_2_513_CA_4_validation,HOUSEHOLD_2_513,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
10756,HOUSEHOLD_2_514_CA_4_validation,HOUSEHOLD_2_514,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0
10757,HOUSEHOLD_2_515_CA_4_validation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


In [8]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Household'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [9]:
len(sales_intermittent)

1493

In [11]:
len(sales_lumpy)

178

In [13]:
len(sales_erratic)

126

In [12]:
len(sales_smooth)

2391

### Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)

In [14]:
sales_intermittent

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
576,HOUSEHOLD_1_012_CA_1_validation,HOUSEHOLD_1_012,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,1,1,0,0,4,0,1,2,1
577,HOUSEHOLD_1_013_CA_1_validation,HOUSEHOLD_1_013,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,2,0,2,2,1,0,4,1
582,HOUSEHOLD_1_018_CA_1_validation,HOUSEHOLD_1_018,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,2,2,2,1,4,1,1,0,3
599,HOUSEHOLD_1_036_CA_1_validation,HOUSEHOLD_1_036,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
600,HOUSEHOLD_1_037_CA_1_validation,HOUSEHOLD_1_037,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10750,HOUSEHOLD_2_508_CA_4_validation,HOUSEHOLD_2_508,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
10751,HOUSEHOLD_2_509_CA_4_validation,HOUSEHOLD_2_509,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,1,2,1,0,0,0,0,0,1
10756,HOUSEHOLD_2_514_CA_4_validation,HOUSEHOLD_2_514,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0
10757,HOUSEHOLD_2_515_CA_4_validation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


# User-defined functions to calculate Metrics and Croston algorithm

In [15]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [16]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [17]:
sales_pattern = sales_intermittent.copy()

In [18]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

# Set value for parameters

In [19]:
list_params = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [20]:
list_params

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9]

In [21]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [22]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [23]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [24]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [25]:
len(sample_list_filtered)

114

# Hyperparameter tuning for Croston

In [26]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [27]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    for i in list_params:
        predictions = list()
        history = [x for x in train_first[num_train_needed*-1:]]
        for t in train_second.index.tolist():
            yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(train_second[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                        'Actual Data': train_second,
                                        'Forecast': predictions,
                                        'Alpha': [i for count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOUSEHOLD_1_432_CA_2
Progressing: 0.0 %
Currently Running: HOUSEHOLD_2_027_CA_2
Progressing: 0.88 %
Currently Running: HOUSEHOLD_2_352_CA_2
Progressing: 1.75 %
Currently Running: HOUSEHOLD_1_205_CA_4
Progressing: 2.63 %
Currently Running: HOUSEHOLD_2_015_CA_4
Progressing: 3.51 %
Currently Running: HOUSEHOLD_2_439_CA_2
Progressing: 4.39 %
Currently Running: HOUSEHOLD_1_435_CA_2
Progressing: 5.26 %
Currently Running: HOUSEHOLD_1_359_CA_2
Progressing: 6.14 %
Currently Running: HOUSEHOLD_2_226_CA_4
Progressing: 7.02 %
Currently Running: HOUSEHOLD_2_082_CA_3
Progressing: 7.89 %
Currently Running: HOUSEHOLD_1_175_CA_4
Progressing: 8.77 %
Currently Running: HOUSEHOLD_1_357_CA_2
Progressing: 9.65 %
Currently Running: HOUSEHOLD_2_253_CA_1
Progressing: 10.53 %
Currently Running: HOUSEHOLD_2_416_CA_3
Progressing: 11.4 %
Currently Running: HOUSEHOLD_1_145_CA_1
Progressing: 12.28 %
Currently Running: HOUSEHOLD_2_039_CA_1
Progressing: 13.16 %
Currently Running: HOUSEHOLD_2_173_CA_

In [28]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOUSEHOLD_1_432_CA_2,2.0,2.620334,0.05
1,HOUSEHOLD_1_432_CA_2,1.0,2.594811,0.05
2,HOUSEHOLD_1_432_CA_2,3.0,2.528609,0.05
3,HOUSEHOLD_1_432_CA_2,8.0,2.548345,0.05
4,HOUSEHOLD_1_432_CA_2,0.0,2.778457,0.05
...,...,...,...,...
16411,HOUSEHOLD_2_173_CA_2,3.0,3.008929,0.90
16412,HOUSEHOLD_2_173_CA_2,2.0,3.000893,0.90
16413,HOUSEHOLD_2_173_CA_2,4.0,2.100089,0.90
16414,HOUSEHOLD_2_173_CA_2,11.0,3.810009,0.90


# Summay all metrics

In [29]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [30]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0749655834666631 minutes


In [31]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_013_CA_4,0.05,1.371555,47.8956,49.5258,40.9301,2.155301,2.936870
1,HOUSEHOLD_1_013_CA_4,0.10,1.446043,50.4968,52.9880,49.2866,2.272354,2.857544
2,HOUSEHOLD_1_013_CA_4,0.15,1.478550,51.6319,53.7286,56.8826,2.323436,2.735642
3,HOUSEHOLD_1_013_CA_4,0.20,1.486067,51.8944,53.3472,61.6858,2.335248,2.667594
4,HOUSEHOLD_1_013_CA_4,0.25,1.480859,51.7125,52.6527,64.4814,2.327064,2.631338
...,...,...,...,...,...,...,...,...
2047,HOUSEHOLD_2_508_CA_3,0.70,3.667103,419.0975,181.5747,41.6455,0.523872,0.525443
2048,HOUSEHOLD_2_508_CA_3,0.75,3.700823,422.9511,180.7157,37.2166,0.528689,0.531989
2049,HOUSEHOLD_2_508_CA_3,0.80,3.691059,421.8353,179.9505,33.0581,0.527294,0.532510
2050,HOUSEHOLD_2_508_CA_3,0.85,3.637831,415.7522,179.3244,29.4938,0.519690,0.526587


# Check what products has unexpected metrics result

In [32]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
162,HOUSEHOLD_1_175_CA_4,0.05,inf,inf,200.0,NaN,2.365222,2.365222
163,HOUSEHOLD_1_175_CA_4,0.10,inf,inf,200.0,NaN,1.980990,1.980990
164,HOUSEHOLD_1_175_CA_4,0.15,inf,inf,200.0,NaN,1.958421,1.958421
165,HOUSEHOLD_1_175_CA_4,0.20,inf,inf,200.0,NaN,1.964159,1.964159
166,HOUSEHOLD_1_175_CA_4,0.25,inf,inf,200.0,NaN,1.966211,1.966211
...,...,...,...,...,...,...,...,...
1975,HOUSEHOLD_2_484_CA_2,0.70,inf,inf,200.0,NaN,4.640416,4.640416
1976,HOUSEHOLD_2_484_CA_2,0.75,inf,inf,200.0,NaN,4.364209,4.364209
1977,HOUSEHOLD_2_484_CA_2,0.80,inf,inf,200.0,NaN,4.082901,4.082901
1978,HOUSEHOLD_2_484_CA_2,0.85,inf,inf,200.0,NaN,3.801833,3.801833


In [33]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
162,HOUSEHOLD_1_175_CA_4,0.05,inf,inf,200.0,NaN,2.365222,2.365222
163,HOUSEHOLD_1_175_CA_4,0.10,inf,inf,200.0,NaN,1.980990,1.980990
164,HOUSEHOLD_1_175_CA_4,0.15,inf,inf,200.0,NaN,1.958421,1.958421
165,HOUSEHOLD_1_175_CA_4,0.20,inf,inf,200.0,NaN,1.964159,1.964159
166,HOUSEHOLD_1_175_CA_4,0.25,inf,inf,200.0,NaN,1.966211,1.966211
...,...,...,...,...,...,...,...,...
1975,HOUSEHOLD_2_484_CA_2,0.70,inf,inf,200.0,NaN,4.640416,4.640416
1976,HOUSEHOLD_2_484_CA_2,0.75,inf,inf,200.0,NaN,4.364209,4.364209
1977,HOUSEHOLD_2_484_CA_2,0.80,inf,inf,200.0,NaN,4.082901,4.082901
1978,HOUSEHOLD_2_484_CA_2,0.85,inf,inf,200.0,NaN,3.801833,3.801833


In [34]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
324,HOUSEHOLD_1_356_CA_2,0.05,1.668727,99.5017,NaN,99.6072,2.860674,5.180307
325,HOUSEHOLD_1_356_CA_2,0.10,1.668123,99.4657,NaN,99.5843,2.859640,5.177924
326,HOUSEHOLD_1_356_CA_2,0.15,1.667451,99.4256,NaN,99.5586,2.858487,5.175267
327,HOUSEHOLD_1_356_CA_2,0.20,1.666696,99.3806,NaN,99.5299,2.857192,5.172285
328,HOUSEHOLD_1_356_CA_2,0.25,1.665842,99.3297,NaN,99.4974,2.855730,5.168917
329,HOUSEHOLD_1_356_CA_2,0.30,1.664870,99.2718,NaN,99.4603,2.854063,5.165081
330,HOUSEHOLD_1_356_CA_2,0.35,1.663752,99.2051,NaN,99.4178,2.852147,5.160673
331,HOUSEHOLD_1_356_CA_2,0.40,1.662454,99.1277,NaN,99.3683,2.849921,5.155555
332,HOUSEHOLD_1_356_CA_2,0.45,1.660927,99.0366,NaN,99.3101,2.847304,5.149538
333,HOUSEHOLD_1_356_CA_2,0.50,1.659105,98.9280,NaN,99.2407,2.844181,5.142366


In [35]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
162,HOUSEHOLD_1_175_CA_4,0.05,inf,inf,200.0,NaN,2.365222,2.365222
163,HOUSEHOLD_1_175_CA_4,0.10,inf,inf,200.0,NaN,1.980990,1.980990
164,HOUSEHOLD_1_175_CA_4,0.15,inf,inf,200.0,NaN,1.958421,1.958421
165,HOUSEHOLD_1_175_CA_4,0.20,inf,inf,200.0,NaN,1.964159,1.964159
166,HOUSEHOLD_1_175_CA_4,0.25,inf,inf,200.0,NaN,1.966211,1.966211
...,...,...,...,...,...,...,...,...
1975,HOUSEHOLD_2_484_CA_2,0.70,inf,inf,200.0,NaN,4.640416,4.640416
1976,HOUSEHOLD_2_484_CA_2,0.75,inf,inf,200.0,NaN,4.364209,4.364209
1977,HOUSEHOLD_2_484_CA_2,0.80,inf,inf,200.0,NaN,4.082901,4.082901
1978,HOUSEHOLD_2_484_CA_2,0.85,inf,inf,200.0,NaN,3.801833,3.801833


In [36]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [37]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [40]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [41]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 3.508771929824561 %
Percentage of unexpected values of WMAPE is: 3.508771929824561 %
Percentage of unexpected values of SMAPE is: 1.7543859649122806 %
Percentage of unexpected values of MAPE is: 3.508771929824561 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [42]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [43]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.214989,130.719988,102.607846,62.573999,2.402508,2.879561
0.10,1.104383,116.618837,96.818691,59.890013,2.122188,2.574841
0.15,1.049981,110.403938,94.567388,60.919601,1.989138,2.419067
0.20,1.023277,106.382556,93.986055,61.564073,1.952114,2.376495
0.25,1.007998,103.404225,93.701998,61.594743,1.949955,2.374957
0.30,0.997871,101.160948,93.487404,61.464223,1.955015,2.382711
0.35,0.990904,99.493929,93.268347,61.326636,1.963950,2.396127
0.40,0.986811,98.313271,93.113728,61.293179,1.977564,2.413943
0.45,0.985628,97.586110,93.113514,61.478221,1.994012,2.435545


In [44]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.45
The optimum Alpha based on Mean WMAPE is: 0.55
The optimum Alpha based on Mean SMAPE is: 0.45
The optimum Alpha based on Mean MAPE is: 0.1
The optimum Alpha based on Mean MAE is: 0.25
The optimum Alpha based on Mean RMSE is: 0.25


### Get MEDIAN metrics value of each Alpha

In [45]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,0.815919,80.52870,97.27695,53.44440,1.354914,1.690989
0.10,0.798521,76.32755,89.30990,48.44135,1.307472,1.662883
0.15,0.792743,74.14740,88.84550,51.73320,1.323477,1.633565
0.20,0.780268,74.62475,86.92550,51.24930,1.322581,1.675976
0.25,0.773217,73.86185,89.12010,50.22890,1.336889,1.682065
0.30,0.782511,75.56050,91.36080,51.05015,1.357542,1.672360
0.35,0.786703,76.73115,88.95565,51.77895,1.338524,1.656834
0.40,0.798561,75.60050,89.18765,51.47995,1.335827,1.652029
0.45,0.801921,75.78355,90.01130,52.43515,1.330636,1.667462


In [46]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.25
The optimum Alpha based on Median of WMAPE is: 0.25
The optimum Alpha based on Median of SMAPE is: 0.2
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.1
The optimum Alpha based on Median of RMSE is: 0.15


# Run Croston model after decide best paramaters

In [47]:
best_alpha = 0.1

In [48]:
dataframe_weekly

,wm_yr_wk,HOUSEHOLD_1_001_CA_2,HOUSEHOLD_1_005_CA_3,HOUSEHOLD_1_006_CA_3,HOUSEHOLD_1_010_CA_2,HOUSEHOLD_1_010_CA_4,HOUSEHOLD_1_012_CA_1,HOUSEHOLD_1_012_CA_3,HOUSEHOLD_1_013_CA_1,HOUSEHOLD_1_013_CA_2,...,HOUSEHOLD_2_514_CA_3,HOUSEHOLD_2_514_CA_4,HOUSEHOLD_2_515_CA_1,HOUSEHOLD_2_515_CA_2,HOUSEHOLD_2_515_CA_3,HOUSEHOLD_2_515_CA_4,HOUSEHOLD_2_516_CA_1,HOUSEHOLD_2_516_CA_2,HOUSEHOLD_2_516_CA_3,HOUSEHOLD_2_516_CA_4
0,11101,0.0,1.0,0.0,0.0,0.0,4.0,0.0,3.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0
1,11102,0.0,29.0,0.0,0.0,0.0,1.0,0.0,6.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0
2,11103,0.0,0.0,0.0,0.0,0.0,4.0,0.0,8.0,6.0,...,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0
3,11104,0.0,10.0,0.0,0.0,0.0,3.0,0.0,5.0,5.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,11105,0.0,17.0,0.0,0.0,0.0,3.0,1.0,6.0,16.0,...,2.0,2.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,0.0,12.0,0.0,0.0,0.0,4.0,0.0,11.0,6.0,...,0.0,2.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0
269,11609,0.0,10.0,0.0,0.0,0.0,1.0,0.0,9.0,3.0,...,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
270,11610,0.0,12.0,0.0,0.0,0.0,4.0,0.0,13.0,0.0,...,2.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0
271,11611,1.0,18.0,0.0,0.0,0.0,6.0,0.0,15.0,0.0,...,1.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0


In [49]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [50]:
df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)

In [51]:
df_result_best_params

,Product,Actual Data,Forecast
0,HOUSEHOLD_1_012_CA_1,6.0,1.799062
1,HOUSEHOLD_1_012_CA_1,8.0,2.051201
2,HOUSEHOLD_1_013_CA_1,15.0,7.975679
3,HOUSEHOLD_1_013_CA_1,6.0,8.645595
4,HOUSEHOLD_1_018_CA_1,0.0,0.000000
...,...,...,...
2981,HOUSEHOLD_2_514_CA_4,0.0,0.769198
2982,HOUSEHOLD_2_515_CA_4,0.0,0.000000
2983,HOUSEHOLD_2_515_CA_4,0.0,0.000000
2984,HOUSEHOLD_2_516_CA_4,0.0,0.738987


In [52]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [53]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_001_CA_2,0.499441,99.8881,168.7498,81.4813,0.499441,0.590678
1,HOUSEHOLD_1_005_CA_3,2.490064,49.8013,65.8792,49.4923,9.960255,10.089178
2,HOUSEHOLD_1_006_CA_3,inf,inf,200.0000,NaN,1.656193,1.656193
3,HOUSEHOLD_1_010_CA_2,NaN,NaN,NaN,NaN,0.000000,0.000000
4,HOUSEHOLD_1_010_CA_4,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...
1488,HOUSEHOLD_2_515_CA_4,NaN,NaN,NaN,NaN,0.000000,0.000000
1489,HOUSEHOLD_2_516_CA_1,0.498927,99.7854,131.7348,48.1798,0.997854,0.998442
1490,HOUSEHOLD_2_516_CA_2,inf,inf,200.0000,NaN,2.075844,2.075844
1491,HOUSEHOLD_2_516_CA_3,0.508784,101.7569,126.2351,41.5655,0.508784,0.517238


In [57]:
df_result_final.isnull().sum()

Product      0
MASE       275
WMAPE      275
SMAPE      277
MAPE       614
MAE          0
RMSE         0
dtype: int64

In [56]:
df_result_final.MAPE.mean()

57.984845961319685

In [58]:
df_result_final.to_csv('Croston_Intermittent_Metrics.csv')